CDM Project: iInsureU123 - k-Anonymity and anonymise the data

 Overview
 In the notebook, we will implement functions to produce a k-anonymous customer information dataset provided by an insurance company, iInsureU123. The anonymised dataset would be made available to the researchers and government. Quasi-identifiers and sensitive attributes of values will be replaced with banded quantities to prevent sampled individuals from being identified. In addition, cryptographic hashing function will be applied to transform certain sensitive data to hash values for privacy reasons. The dataset will be made available to public by government whereas researchers from Imperial College will use dataset for research purposes. Therefore, two dataset containing different attributes will be generated and encrypted.

In the project, we will use the Pandas to load dataset as a data frame. Other modules will be used have been listed as below.

In [27]:
# Packages loading

import pandas as pd
import datetime
import matplotlib.pyplot as plt
from datetime import date
import hashlib
import random, string

Reading the raw data "customer_information" in csv as a dataframe with Pandas

In [28]:
# Import data and target
df = pd.read_csv('customer_information.csv') #change to own source's directory path

The dataset are used for research on the association of the gene DRD4 and travelling from educational or geographical perspective. However, individual height and weight may be not helpful for the research purpose. We decide to convert height and weight into BMI for both research purposes and privacy-preserving reason.

In [29]:
# convert just colums "weight" and "height" into numeric.
df["weight"] = pd.to_numeric(df["weight"])
df["height"] = pd.to_numeric(df["height"])
df['heightsquared']=df['height']**2

#df.insert(1,"BMI",[])
df['bmi']=df['weight']/df['heightsquared']
df['bmi']=df['bmi'].apply(lambda x:round(x,2))

# delete the column 'weight','height'and 'heightsquared'
del df['weight']
del df['height']
del df['heightsquared']

In the following section, values of quasi identifiers and sensitive attributes in k-anonymous group will be banded. For example, birthdate will be converted to banded age groups; participants will be categorized into light, intermediate and heavy smokers according to average cigarettes smoked per week; consumption of drinks will be categorized to low-level, moderate and high-level alcoholic consumption; participants will be grouped to "underweight","healthy" and "overweight"; countries visited will be banded into 3 groups.

In [67]:
# Generate a function to convert birthdate to age group
# Years of birth
birthdate=pd.to_datetime(df.birthdate)
df['birth_year'] = pd.DatetimeIndex(birthdate).year

# Age
def from_birthdate_to_age(birth_date):
    now=pd.Timestamp('now')
    now_year,now_month,now_day = now.year, now.month, now.day
    birth_date = pd.to_datetime(birth_date)
    birth_year, birth_month, birth_day = birth_date.year, birth_date.month, birth_date.day
    age = now_year - birth_year
    if now_month >= birth_month:
        if now_day >= birth_day:
            age = now_year - birth_year + 1
    return (age)

df['age'] = df['birthdate'].apply(from_birthdate_to_age)

# Banding age to groups
bins_age = [18,27,37,47,57,67]
labels_age= ['18-27','28-37','38-47','48-57','58-67']
df['age_groups'] = pd.cut(df.age, bins = bins_age, labels = labels_age)

# Banding age to groups
bins_age_gov = [18,37,58,67]
labels_age_gov= ['18-38','39-58','59-68']
df['age_groups_gov'] = pd.cut(df.age, bins = bins_age_gov, labels = labels_age_gov)

In [68]:
# Banding smoking per week
## we are defining smoking <=40 cig per week as light smokers, 40-175 as intermediate smokers, >175 as heavy smokers [(Schane et al., 2010)][(Wilson et al., 1992)]
bins_smok= [0, 40, 175, 500]
labels_smok=['light smokers','intermediate smokers','heavy smokers']
df['smoking_status'] = pd.cut(df.avg_n_cigret_per_week, bins = bins_smok, labels = labels_smok)

# Banding avg_n_drinks_per_week
## 0-3.9 as low-level alc consumption, 4-6.9 as moderate alc consumption,7-10 as high-level alc consumption
bins_alc = [0,4,7,10]
label_alc = ['low', 'moderate', 'high']
df['level of drinking_status'] = pd.cut(df.avg_n_drinks_per_week, bins = bins_alc, labels = label_alc, right = False)


In [69]:
# Banding 'BMI' to groups
## 0-18.5 as underweight, 18.5-24.9 as healthy,24.9 and over as overweight

bins_bmi = [0,18.5,24.9,50]
label_bmi = ['Underweight','Healthy','Overweight']
df['level of bmi'] = pd.cut(df.bmi, bins = bins_bmi, labels = label_bmi, right = False)

# Banding countries visited to groups
bins_countries = [1, 18, 34, 50]
labels_countries= ['2-18','19-34','35-50']
df['countries_visited'] = pd.cut(df.n_countries_visited, bins = bins_countries, labels = labels_countries)

The government aims to identify the association of genes and educational or geographical background. Therefore,country of birth and education levels are essential for government. In the following section, country of birth will be grouped by continents and cultural reasons because geographical and socio-cultural perspectives affect the decisions on migration.

In [70]:
# Country_of_birth Clearing Up
import pycountry_convert as pc  #pip install pycountry-convert
df['country_of_birth'] = df['country_of_birth'].replace(['Korea'], 'South Korea')
df['country_of_birth'] = df['country_of_birth'].replace(['Palestinian Territory'], 'Jordan')
df['country_of_birth'] = df['country_of_birth'].replace(['Saint Barthelemy'], 'Dominican Republic')
df['country_of_birth'] = df['country_of_birth'].replace(['Saint Helena'], 'South Africa')
df['country_of_birth'] = df['country_of_birth'].replace(['Reunion'], 'Mauritius')
df['country_of_birth'] = df['country_of_birth'].replace(['Svalbard and Jan Mayen'], 'Greenland')
df['country_of_birth'] = df['country_of_birth'].replace(['United States Minor Outlying Islands'], 'United States')
df['country_of_birth'] = df['country_of_birth'].replace(['Antarctica (the territory South of 60 deg S)'], 'Heard Island and McDonald Islands')
df['country_of_birth'] = df['country_of_birth'].replace(['Western Sahara'], 'Morocco')
df['country_of_birth'] = df['country_of_birth'].replace(['Svalbard & Jan Mayen Islands'], 'Heard Island and McDonald Islands')
df['country_of_birth'] = df['country_of_birth'].replace(['Libyan Arab Jamahiriya'], 'Libya')
df['country_of_birth'] = df['country_of_birth'].replace(['Pitcairn Islands'], 'Fiji')
df['country_of_birth'] = df['country_of_birth'].replace(['Slovakia (Slovak Republic)'], 'Slovakia')
df['country_of_birth'] = df['country_of_birth'].replace(['Bouvet Island (Bouvetoya)'], 'Heard Island and McDonald Islands')
df['country_of_birth'] = df['country_of_birth'].replace(['Holy See (Vatican City State)'], 'Italy')
df['country_of_birth'] = df['country_of_birth'].replace(['Timor-Leste'], 'Indonesia')
df['country_of_birth'] = df['country_of_birth'].replace(["Cote d'Ivoire"], 'Ghana')
df['country_of_birth'] = df['country_of_birth'].replace(['British Indian Ocean Territory (Chagos Archipelago)'], 'India')
df['country_of_birth'] = df['country_of_birth'].replace(["Netherlands Antilles"], "Netherlands")

# Group original countries to continent by applying a function to dataframe
def country_to_continent(country_of_birth):
    country_alpha2 = pc.country_name_to_country_alpha2(country_of_birth)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name
df['continent_of_birth'] = df['country_of_birth'].apply(country_to_continent)

# Group some continents according to geographic / cultural reasons
df['continent_of_birth'] = df['continent_of_birth'].replace(['North America'], 'America')
df['continent_of_birth'] = df['continent_of_birth'].replace(['South America'], 'America')
df['continent_of_birth'] = df['continent_of_birth'].replace(['Antarctica'], 'Europe')

In the following section, we band the education level into three group. Primary and secondary education will be grouped to pre-university whereas masters and phD will be identified as post-graduate qualification. As apprenticeship or diploma are not included in the list, we assume that other degree could contain these academic qualification which then should be categorized as bachelor.

In [71]:
# Banding
df['education_level'] = df['education_level'].replace(['secondary'],'pre-uni')
df['education_level'] = df['education_level'].replace(['primary'], 'pre-uni')
df['education_level'] = df['education_level'].replace(['bachelor'], 'bachelor')
df['education_level'] = df['education_level'].replace(['other'], 'bachelor')
df['education_level'] = df['education_level'].replace(['masters'], 'post-grad')
df['education_level'] = df['education_level'].replace(['phD'], 'post-grad')

In the section below, we will create pseudonyms to anonymize direct sensitive identifiers (e.g. names, national insurance numbers). Secure Hashing Algorithm (SHA) will be used to map personal identifiers into a non-identifiable strings. In addition to SHA, we also add a salt i.e. random number generated by the computer to enhance the security.

In [72]:
#SHA ----------------------------------------------------------------

#Salt generator
def randomword(length):
   letters = string.ascii_lowercase #generates lowercase letters
   return ''.join(random.choice(letters) for i in range(length)) #generates salt from letters

df['NI_enc']=df['national_insurance_number'].str.encode('utf-8') #utf encoding needed for sha function

key='password123'.encode('utf-8') #encoding key

#Hash function
hashes=[]
salt=[]
for i in range(len(df['NI_enc'])):
    salt.append(randomword(10).encode('utf-8'))
    hashes.append(hashlib.sha1(key+salt[i]+df['NI_enc'][i]).hexdigest()) #hash function applied 

df['hash']=hashes
df['postcode_split'] = df['postcode'].str.split().str[0] 
hash_cols=['given_name','surname','phone_number','national_insurance_number','blood_group','postcode','birthdate','age','bank_account_number','birth_year','avg_n_drinks_per_week','avg_n_cigret_per_week','n_countries_visited','bmi','country_of_birth']

#Lookup table generation
secure_df=df[hash_cols]
secure_df['hash']=df['hash']
secure_df['salt']=salt
new_cust=df.drop(columns=hash_cols)
new_cust=new_cust.drop(columns='NI_enc')

C:\Users\jay_g\AppData\Local\Temp\ipykernel_12592\2780082543.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secure_df['hash']=df['hash']
C:\Users\jay_g\AppData\Local\Temp\ipykernel_12592\2780082543.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  secure_df['salt']=salt


K-anonymity is computed based on processed data in order to secure the dataset's confidentiality in information sharing in public. In the situation, the database is said to become K Anonymous. In the following sectioin, we calculate the k-anonminity for both researchers at Imperial College and government. Due to the difference in research purposes, dataset will contain different attributes as discussed previously. As a result. quasi identifiers differ in each anonymised dataset.

In [73]:
# K-anonyminity - Currently freezes when adding any more quasi_identifiers
quasi_identifiers_imp=['age_groups','level of bmi','gender']
k_df_imp=new_cust.groupby(quasi_identifiers_imp,observed=True).size().reset_index(name='Count').sort_values(by='Count')
print(f'The Imperial K-anonyminity is: {min(k_df_imp["Count"])}')

# K-anonyminity - Currently freezes when adding any more quasi_identifiers
quasi_identifiers_gov=['level of bmi','continent_of_birth','education_level']
k_df_gov=new_cust.groupby(quasi_identifiers_gov,observed=True).size().reset_index(name='Count').sort_values(by='Count')
print(f'The Government K-anonyminity is: {min(k_df_gov["Count"])}')

The Imperial K-anonyminity is: 17
The Government K-anonyminity is: 6


Here, we will export anonymised dataframe to CSV File.

In [74]:
# convert the dataframe into csv
new_cust.to_csv('new_cust.csv')